In [3]:
import os
import re
import json
import requests
from bs4 import BeautifulSoup, Tag
from urllib.parse import urljoin, unquote

BASE_URL = 'https://zh.wikipedia.org'
MAIN_URL = BASE_URL + '/wiki/Wikipedia:%E6%96%B0%E6%9D%A1%E7%9B%AE%E6%8E%A8%E8%8D%90'

OUTPUT_DIR = 'data/cn'


def get_month_links_from_archive(main_page_url):
    resp = requests.get(main_page_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')

    container = soup.find('div', class_='center')
    if not container:
        raise ValueError("Could not find the container div with class 'center'.")
    table = container.find('table', class_='ombox ombox-notice mbox-small')
    if not table: 
        raise ValueError("Could not find the table with class 'ombox ombox-notice mbox-small' in the container div.")
    main_row = table.find_all('tr')[1]
    if not main_row:
        raise ValueError("Could not find the main row in the table.")
    row_table = main_row.find('table')
    if not row_table:
        raise ValueError("Could not find the row table in the main row.")

    archive = {}
    for tr in row_table.find_all('tr'):
        th = tr.find('th')
        if not th:
            continue
        year = th.text.strip()[:-1]
        if not re.match(r'^\d{4}$', year):
            continue
        
        months = []
        for td in tr.find_all('td'):
            a = td.find('a', href=True)
            if not a:
                continue

            month_name = a.text.strip()
            href = a['href']
            full_url = urljoin(BASE_URL, href)
            exists = 'new' not in a.get('class', [])
            months.append({
                'month': month_name,
                'url': full_url,
                'exists': exists
            })
        if months:
            archive[year] = months
        
    return archive    

In [4]:
archive = get_month_links_from_archive(MAIN_URL)

archive

{'2005': [{'month': '3',
   'url': 'https://zh.wikipedia.org/wiki/Wikipedia:%E6%96%B0%E6%9D%A1%E7%9B%AE%E6%8E%A8%E8%8D%90/2005%E5%B9%B43%E6%9C%88',
   'exists': True},
  {'month': '4',
   'url': 'https://zh.wikipedia.org/wiki/Wikipedia:%E6%96%B0%E6%9D%A1%E7%9B%AE%E6%8E%A8%E8%8D%90/2005%E5%B9%B44%E6%9C%88',
   'exists': True},
  {'month': '5',
   'url': 'https://zh.wikipedia.org/wiki/Wikipedia:%E6%96%B0%E6%9D%A1%E7%9B%AE%E6%8E%A8%E8%8D%90/2005%E5%B9%B45%E6%9C%88',
   'exists': True},
  {'month': '6',
   'url': 'https://zh.wikipedia.org/wiki/Wikipedia:%E6%96%B0%E6%9D%A1%E7%9B%AE%E6%8E%A8%E8%8D%90/2005%E5%B9%B46%E6%9C%88',
   'exists': True},
  {'month': '7',
   'url': 'https://zh.wikipedia.org/wiki/Wikipedia:%E6%96%B0%E6%9D%A1%E7%9B%AE%E6%8E%A8%E8%8D%90/2005%E5%B9%B47%E6%9C%88',
   'exists': True},
  {'month': '8',
   'url': 'https://zh.wikipedia.org/wiki/Wikipedia:%E6%96%B0%E6%9D%A1%E7%9B%AE%E6%8E%A8%E8%8D%90/2005%E5%B9%B48%E6%9C%88',
   'exists': True},
  {'month': '9',
   'url': 'http

In [5]:
def parse_month_facts(month_url: str) -> list[dict]:
    resp = requests.get(month_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    results: list[dict] = []

    div = soup.find("div", class_="mw-heading mw-heading2 ext-discussiontools-init-section")
    if not div:
        raise ValueError("Could not find the div with class 'mw-heading mw-heading2 ext-discussiontools-init-section'.")
    div_siblings = div.find_next_siblings()
    section_title = div_siblings[0].get_text(strip=True)

    for elem in div_siblings[1:]:
        if not (isinstance(elem, Tag) and elem.name == "ul"):
            continue

        for li in elem.find_all("li")[:]:
            fact_text = li.get_text(" ", strip=True)
            
            links = []
            relevant_links = []
            for a in li.find_all("a", href=True):
                href = a["href"]
                if not href.startswith("/wiki/"):
                    continue
            
                full_url = unquote(urljoin(month_url, href))

                if a.find_parent('b'):
                    relevant_links.append(full_url)

                links.append(full_url)

            results.append({
                "section": section_title,
                "text": fact_text,
                "links": links,
                "relevant_links": relevant_links,
            })

    return results

In [12]:
year = '2025'
month = '7'
month_url = archive[year][int(month) - 1]['url']

facts = parse_month_facts(month_url)
facts

[{'section': '2025年7月',
  'text': '哪位英國君主 在位63年216天，並在1876年獲得“ 印度女皇 ”頭銜，更見證了 大英帝國 的大規模擴張？',
  'links': ['https://zh.wikipedia.org/wiki/维多利亚女王',
   'https://zh.wikipedia.org/wiki/印度皇帝',
   'https://zh.wikipedia.org/wiki/大英帝國'],
  'relevant_links': ['https://zh.wikipedia.org/wiki/维多利亚女王']},
 {'section': '2025年7月',
  'text': '哪一部动画电影 是基于 安田现象 第一部动画短片改编，于2025年1月上映，并且制作过程中两次众筹都以超出目标金额两倍而完成制作资金筹集？',
  'links': ['https://zh.wikipedia.org/wiki/Make_A_Girl',
   'https://zh.wikipedia.org/wiki/安田现象'],
  'relevant_links': ['https://zh.wikipedia.org/wiki/Make_A_Girl']},
 {'section': '2025年7月',
  'text': '哪位人物 曾任延吉县游击大队长和东北人民革命第二军独立师第一团团长，后被中共错杀？',
  'links': ['https://zh.wikipedia.org/wiki/朴东根'],
  'relevant_links': ['https://zh.wikipedia.org/wiki/朴东根']},
 {'section': '2025年7月',
  'text': '日本 哪個公家 有 當主 （ 日语 ： 山科言継 ） 的 日記 因載其妻患有 疟疾 時的病情的 病历 而被 服部敏良 （ 日语 ： 服部敏良 ） 贊譽為「 医学史 上極為珍貴的史料」？',
  'links': ['https://zh.wikipedia.org/wiki/日本',
   'https://zh.wikipedia.org/wiki/山科家',
   'https://zh.wikipedia.org/

In [13]:
import dateparser

year = '2025'
month = '7'

dt_month = dateparser.parse(f"{month} {year}")
fact_date = dt_month.strftime("%Y-%m") if dt_month else None

fact_date

'2025-07'

In [14]:
import spacy

model_news = f"zh_core_news_sm"
model_web  = f"zh_core_web_sm"

try:
    nlp = spacy.load(model_news)
except OSError:
    try:
        nlp = spacy.load(model_web)
    except OSError:
        print(f"Neither {model_news} nor {model_web} found. Downloading {model_news}…")

Neither zh_core_news_sm nor zh_core_web_sm found. Downloading zh_core_news_sm…
